In [11]:
# import libraries

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [12]:
# each team has a code number to refer

club_id_list_2022 = [2,591,10,17,16,3,592,5,8,28,57,15,9,25,4,12,14,13] # code number for each club website 13 = VBC
club_name_list_2022 = ['BAR','GIR','BAX','FUE','CAZ','BKN','COV','GCA','JOV','LNT','MOB','BET','RMB','BRE','SBB','UCM','UNI','VBC']

In [13]:
def extract_player_stats(stats_table,num_player):
    # extract player stats from stats table
    
    player = []

    player_stats = stats_table.find_all('tr')[num_player].find_all('td')
    player.append(player_stats[1].string)
    player.append(club_name)

    for td in player_stats[2:]:
        td.string = td.string.replace(':','.')
        td.string = td.string.replace(',','.')
        td.string = td.string.replace('%','')
        player.append(float(td.string))
    
    return player

In [14]:
def extract_team_stats(stats_table,num_team):
    # extract team stats from stats table
    
    team = []

    team_stats_table = stats_table.find_all('tr')[num_team].find_all('td')
    team.append(club_name)

    for td in team_stats_table[2:]:
        td.string = td.string.replace(':','.')
        td.string = td.string.replace(',','.')
        td.string = td.string.replace('%','')
        
        # check empty cells
        if td.string.strip() == '':
            team.append(np.nan)
        else:
            team.append(float(td.string))
    
    return team

In [17]:
cols_player = ['PLAYER','TEAM','GM','MIN','5i','PTS','3PM','3PA','3P%','2PM','2PA','2P%','FTM','FTA','FT%','DREB','OREB','TREB','AS','ST','TO','BL','BA','DU','FR','FA','+/-','VAL']
cols_team = ['TEAM','GM','MIN','5i','PTS','3PM','3PA','3P%','2PM','2PA','2P%','FTM','FTA','FT%','DREB','OREB','TREB','AS','ST','TO','BL','BA','DU','FR','FA','+/-','VAL']

stats = pd.DataFrame(columns=cols_player) # create dataframe with defined columns
team_stats = pd.DataFrame(columns=cols_team) # create team stats dataframe

stats_type = 0   # type of stats to extract 0 = totals // 1 = average

for club_id in club_id_list_2022:
        
    # get html code using BeautifulSoup
    html_doc = urlopen('https://acb.com/club/estadisticas/id/'+str(club_id)+'/temporada_id/2022/edicion_id/967/fase_id/0/tipo_id/'+str(stats_type)).read() # total stats
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    # find stats table in html
    stats_table = soup.find('tbody')
    
    club_name = club_name_list_2022[club_id_list_2022.index(club_id)] # search club name by id
    print(club_name)
    
    num_players_by_team = len(stats_table.find_all('span')) # number of players by team
    
    for num_player in range(num_players_by_team):
        
        player = extract_player_stats(stats_table,num_player) # extract player stats
        
        stats.loc[len(stats)] = player # append player to dataframe
    
    team = extract_team_stats(stats_table,num_players_by_team)
    team_stats.loc[len(team_stats)] = team

BAR
GIR
BAX
FUE
CAZ
BKN
COV
GCA
JOV
LNT
MOB
BET
RMB
BRE
SBB
UCM
UNI
VBC


In [18]:
stats.set_index('PLAYER',inplace=True) # set player names as index
team_stats.set_index('TEAM',inplace=True) # set team names as index

In [19]:
stats.head()

,TEAM,GM,MIN,5i,PTS,3PM,3PA,3P%,2PM,2PA,...,AS,ST,TO,BL,BA,DU,FR,FA,+/-,VAL
PLAYER,,,,,,,,,,,,,,,,,,,,,
Nico Laprovittola,BAR,22.0,504.54,21.0,308.0,55.0,121.0,45.5,46.0,93.0,...,83.0,18.0,50.0,0.0,9.0,0.0,42.0,66.0,172.0,307.0
Rokas Jokubaitis,BAR,23.0,449.37,1.0,198.0,19.0,46.0,41.3,51.0,105.0,...,101.0,18.0,37.0,2.0,7.0,4.0,32.0,47.0,126.0,248.0
Jan Vesely,BAR,22.0,417.22,12.0,211.0,0.0,1.0,0.0,90.0,146.0,...,26.0,20.0,27.0,2.0,7.0,23.0,47.0,45.0,110.0,240.0
Tomas Satoransky,BAR,21.0,438.38,21.0,156.0,15.0,42.0,35.7,40.0,68.0,...,75.0,20.0,33.0,0.0,4.0,3.0,46.0,44.0,124.0,203.0
Nikola Kalinic,BAR,22.0,414.01,13.0,138.0,12.0,39.0,30.8,33.0,60.0,...,33.0,12.0,28.0,1.0,3.0,5.0,32.0,44.0,135.0,186.0


In [20]:
team_stats.head()

,GM,MIN,5i,PTS,3PM,3PA,3P%,2PM,2PA,2P%,...,AS,ST,TO,BL,BA,DU,FR,FA,+/-,VAL
TEAM,,,,,,,,,,,,,,,,,,,,,
BAR,23.0,NaN,NaN,1.938,215.0,586.0,36.7,474.0,860.0,55.1,...,430.0,213.0,291.0,39.0,61.0,67.0,451.0,474.0,NaN,2.271
GIR,23.0,NaN,NaN,1.750,156.0,507.0,30.8,488.0,953.0,51.2,...,385.0,252.0,329.0,67.0,57.0,73.0,418.0,431.0,NaN,1.980
BAX,22.0,NaN,NaN,1.803,189.0,569.0,33.2,478.0,916.0,52.2,...,394.0,209.0,331.0,53.0,54.0,54.0,522.0,440.0,NaN,1.803
FUE,23.0,NaN,NaN,1.773,156.0,493.0,31.6,493.0,950.0,51.9,...,369.0,215.0,339.0,49.0,72.0,53.0,467.0,439.0,NaN,1.800
CAZ,23.0,NaN,NaN,1.804,157.0,499.0,31.5,501.0,954.0,52.5,...,362.0,232.0,345.0,107.0,50.0,87.0,466.0,488.0,NaN,2.041


In [21]:
# save dataframes to csv

if stats_type == 0:
    stats.to_csv('ACB_2022_player_total.csv') # save total stats dataframe to csv file
    team_stats.to_csv('ACB_2022_team_total.csv')
elif stats_type == 1:
    stats.to_csv('ACB_2022_player_avg.csv') # save average stats dataframe to csv file
    team_stats.to_csv('ACB_2022_team_avg.csv')